In [1]:
import sys
import glob
import imp
from pathlib import Path
import itertools
import pandas as pd
import mdtraj as md
import logging
import plotly.graph_objects as go
import plotly.express as px
import plotly.express as px
import plotly.io as pio
px.defaults.template = 'ggplot2'
px.defaults.template = 'simple_white'
pio.templates.default = 'simple_white'
cdr_colors = ["#74D2F5", "#5495D6", "#3763F5", "#CF81EB", "#E564F5", "#A04AD6",
    'Aqua', 'AquaMarine']
from collections import Counter
import pickle
import subprocess
import freesasa
from joblib import Parallel, delayed
import importlib
source_location = Path().resolve()
sys.path.append(source_location)

from abag_interactions_rings import *
from interactions_polar import *
from names import *

casa_dir = Path("/home/pbarletta/labo/22/abag")
data_dir = Path.joinpath(casa_dir, "data")
str_dir = Path.joinpath(casa_dir, "structures/raw")
exposed_dir = Path.joinpath(casa_dir, "structures/exposed")

In [2]:
print("Reading epitope_buried_cleaned.pickle")
with open(Path.joinpath(
        casa_dir, "data", 'epitope_buried_cleaned.pickle'), 'rb') as file:
    epitope_buried_cleaned = pickle.load(file)

print("Reading buried_interface_res.pickle")
with open(Path.joinpath(casa_dir, "data", 'buried_interface_res.pickle'), 'rb') as file:
    buried_interface_res = pickle.load(file)

with open(Path.joinpath(data_dir, 'pdb.list'), 'r') as file:
    pdb_list = [ linea.strip() for linea in file ]

with open(Path.joinpath(data_dir, 'interface_atoms.pkl'), 'rb') as file:
        interface_atoms = pickle.load(file)

Reading epitope_buried_cleaned.pickle
Reading buried_interface_res.pickle


----

## Hydrophobic

In [7]:
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with(open(Path.joinpath(data_dir, 'pdb.list'), 'r')) as file:
    pdb_list = [ linea.strip() for linea in file ]
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)
with open(Path.joinpath(data_dir, 'interface_atoms.pkl'), 'rb') as file:
    interface_atoms = pickle.load(file)
with open(Path.joinpath(data_dir, 'hydrophobic.pkl'), 'rb') as file:
    hydrophobic = pickle.load(file)

In [4]:
import abag_interactions_hydrophobic
imp.reload(abag_interactions_hydrophobic)
from abag_interactions_hydrophobic import *

In [49]:
check_pdb = '5vk2'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
# for pdb_idcode in pdb_list:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody 
    ag_chains = chains[pdb_idcode].antigen
    #
    # Hydrophobic clusters
    #
    try:
        G = get_carbons_graph(trj_in, interface_atoms[pdb_idcode], cutoff_carbons)
        pre_clusteres = get_putative_clusters(G)
        clusters = merge_clusters(trj_in, pre_clusteres, cutoff_clusters)
    except Exception as e:
        logging.warning(
            f"- {pdb_idcode} raised: {e.__class__}, saying: {e}, during hydrophobic "
            f"interactions calculation. Probably has no hydrophobic interactions.")
        raise e

In [50]:
draw_clusters(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename),
    interface_atoms[pdb_idcode], ag_chains, clusters,
    str( Path.joinpath(casa_dir, 'aux', "clusters.py")))

In [285]:
cadenas = []
for i, row in enumerate(epitope_buried_cleaned.query(f"idcode == '{check_pdb}'").iterrows()):
    beg = int(row[1].cdr_begin)
    end = int(row[1].cdr_end) + 1
    chainID = row[1].chainID
    chain_type = row[1].chain_type
    resis = range(beg, end)
    cadena = chain_type + str(i%3+1)
    cadenas.append(cadena)

    print(f"cdr{cadena} = '", end='')
    for resi in resis[:-1]:
        print(f"resi {resi} and chain {chainID}", end = ' or ')
    print(f"resi {resi} and chain {chainID}'")

cdrH1 = 'resi 26 and chain H or resi 27 and chain H or resi 28 and chain H or resi 29 and chain H or resi 30 and chain H or resi 31 and chain H or resi 31 and chain H'
cdrH2 = 'resi 52 and chain H or resi 53 and chain H or resi 54 and chain H or resi 55 and chain H or resi 56 and chain H or resi 56 and chain H'
cdrH3 = 'resi 99 and chain H or resi 100 and chain H or resi 101 and chain H or resi 102 and chain H or resi 103 and chain H or resi 104 and chain H or resi 105 and chain H or resi 106 and chain H or resi 107 and chain H or resi 108 and chain H or resi 109 and chain H or resi 110 and chain H or resi 111 and chain H or resi 112 and chain H or resi 113 and chain H or resi 114 and chain H or resi 114 and chain H'
cdrL1 = 'resi 23 and chain L or resi 24 and chain L or resi 25 and chain L or resi 26 and chain L or resi 27 and chain L or resi 28 and chain L or resi 29 and chain L or resi 30 and chain L or resi 31 and chain L or resi 32 and chain L or resi 32 and chain L'
cdrL2 = 'resi

In [217]:
with open(Path.joinpath(casa_dir, 'data', 'shielding.pkl'), 'rb') as file:
    shielding_dict = pickle.load(file)

### Interface CDRs vs fr

In [20]:
with open(Path.joinpath(data_dir, '2interface_atoms.pkl'), 'rb') as file:
        interface_atoms2 = pickle.load(file)

In [21]:
# Check how many atoms belong to framework.
cola = []
for pdb_idcode in interface_atoms2.keys():
    cderes = [atm.CDR for atm in interface_atoms2[pdb_idcode].antibody.values()]
    cola.append(cderes.count(0) / len(cderes))
px.histogram(cola)

In [22]:
with open(Path.joinpath(data_dir, 'interface_atoms.pkl'), 'rb') as file:
        interface_atoms = pickle.load(file)

In [24]:
# Check how many atoms belong to framework heavy chain.
cola = []
for pdb_idcode in interface_atoms.keys():
    cderes = [atm.CDR for atm in interface_atoms[pdb_idcode].antibody.values()]
    cola.append(cderes.count(0) / len(cderes))
# print(suma / len(interface_atoms.keys()))
px.histogram(cola)

In [94]:
# Check how many atoms belong to framework heavy chain.
cola = []
for pdb_idcode in interface_atoms.keys():
    cderes = [atm.CDR for atm in interface_atoms[pdb_idcode].antibody.values()]
    if (cderes.count(0) / len(cderes)) > .9:
        print(filenames[pdb_idcode])

5u5m_complex_BA_CE.pdb
6w7s_complex_HL_A.pdb
7kpj_complex_AB_E.pdb
7lu9_complex_lk_f.pdb
7rxc_complex_HL_B.pdb


## Pi-Pi

In [179]:
import abag_interactions_rings
imp.reload(abag_interactions_rings)
from abag_interactions_rings import *

In [175]:
PiPi = {}
check_pdb = '5t1d'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen
    PiPi[pdb_idcode] = tuple()

    try:
        CG_rings, CoM_rings_xyz, normal_vectors = get_ring_data(
            trj_in, ab_chains, ring_atoms)
        pipi_ring_pairs = get_pipi_interactions(
            trj_in, CG_rings, CoM_rings_xyz, normal_vectors, cutoff_ring,
            cutoff_angle_pipi)

        rings = get_data_from_ring_ring(pdb_idcode, epitope_buried_cleaned, pipi_ring_pairs)
    except Exception as e:
        logging.error(
            f"- {pdb_idcode} raised: {e.__class__}, saying: {e}, during PiPi "
            f"interactions calculation.")
        raise e
    else:
        PiPi[pdb_idcode] = rings

In [176]:
PiPi

{'5t1d': ()}

In [177]:

CG_rings

{'antibody': [1899,
  1934,
  1980,
  2047,
  2229,
  2313,
  2409,
  2430,
  2502,
  2727,
  2944,
  2965,
  3027,
  3087,
  3107,
  3370,
  3689,
  3710,
  3833,
  3964,
  3995,
  4133,
  4316,
  4474,
  5125,
  5202,
  5242,
  5259,
  5283,
  5506,
  5614,
  5718,
  5818,
  6055,
  6076,
  6164,
  6249,
  6541,
  6764,
  6831,
  6851,
  6991,
  7223,
  7372,
  7572,
  7632,
  7674,
  7758,
  7929],
 'antigen': [39, 240, 380, 745, 765, 982, 1190, 1229, 1322, 1430, 1461, 1485]}

In [171]:
with open(Path.joinpath(data_dir, 'PiPi.pkl'), 'rb') as file:
        PiPi = pickle.load(file)

In [10]:
PiPi['4oqt'][0].antibody

Ring(indices=(5842, 5843, 5844, 5845, 5846, 5847, 5848, 5849, 5850, 5851, 5852), serials=(5845, 5846, 5847, 5848, 5849, 5850, 5851, 5852, 5853, 5854, 5855), resSeq=57, resSeq_str='', resname='PHE', chain_ID='H', chain_type='H', CDR=2)

In [11]:
PiPi['4oqt'][0].antigen

Ring(indices=(937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948), serials=(938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949), resSeq=122, resSeq_str='', resname='TYR', chain_ID='A', chain_type='.', CDR=-1)

In [69]:
a = tuple(interface_atoms[pdb_idcode].antigen.values())

In [97]:
type((*(1, 2), *(3, 4)))

tuple

### Pi-Ion

In [65]:
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with(open(Path.joinpath(data_dir, 'pdb.list'), 'r')) as file:
    pdb_list = [linea.strip() for linea in file]
with open(Path.joinpath(data_dir, 'interface_atoms.pkl'), 'rb') as file:
    interface_atoms = pickle.load(file)
print("Starting now.")

Starting now.


In [178]:
import abag_interactions_rings
imp.reload(abag_interactions_rings)
from abag_interactions_rings import *

In [8]:
PiAnion = {}
PiCation = {}
check_pdb = '5i5k'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen
    PiAnion[pdb_idcode] = tuple()
    PiCation[pdb_idcode] = tuple()

    try:
        CG_rings, CoM_rings_xyz, normal_vectors = get_ring_data(
            trj_in, ab_chains, ring_atoms_pi_ion)

        ab_anions, ag_anions, ab_cations, ag_cations = get_ions(
            interface_atoms[pdb_idcode])

        ring_ab_anion_ag = get_ion_ring_interactions(
            trj_in, CG_rings["antibody"], ag_anions, CoM_rings_xyz["antibody"],
            normal_vectors["antibody"], trj_in.xyz[0], cutoff_ring, cutoff_angle_pion)
        ring_ab_cation_ag = get_ion_ring_interactions(
            trj_in, CG_rings["antibody"], ag_cations, CoM_rings_xyz["antibody"],
            normal_vectors["antibody"], trj_in.xyz[0], cutoff_ring, cutoff_angle_pion)

        ring_ag_anion_ab = get_ion_ring_interactions(
            trj_in, CG_rings["antigen"], ab_anions, CoM_rings_xyz["antigen"],
            normal_vectors["antigen"], trj_in.xyz[0], cutoff_ring, cutoff_angle_pion)

        ring_ag_cation_ab = get_ion_ring_interactions(
            trj_in, CG_rings["antigen"], ab_cations, CoM_rings_xyz["antigen"],
            normal_vectors["antigen"], trj_in.xyz[0], cutoff_ring, cutoff_angle_pion)

        data_ring_ab_anion_ag = get_data_from_ring_ab_ion_ag(
            pdb_idcode, epitope_buried_cleaned, ring_ab_anion_ag)

        data_ring_ab_cation_ag = get_data_from_ring_ab_ion_ag(
            pdb_idcode, epitope_buried_cleaned, ring_ab_cation_ag)

        data_ring_ag_anion_ab = get_data_from_ring_ag_ion_ab(
            pdb_idcode, epitope_buried_cleaned, ring_ag_anion_ab)

        data_ring_ag_cation_ab = get_data_from_ring_ag_ion_ab(
            pdb_idcode, epitope_buried_cleaned, ring_ag_cation_ab)

    except Exception as e:
        logging.error(
            f"- {pdb_idcode} raised: {e.__class__}, saying: {e}, during Pi-ion "
            f"interactions calculation. Aborting.")
        raise e
    else:
        PiAnion[pdb_idcode] = (*data_ring_ab_anion_ag, *data_ring_ag_anion_ab)
        PiCation[pdb_idcode] = (*data_ring_ab_cation_ag, *data_ring_ag_cation_ab)

In [77]:
with open(Path.joinpath(data_dir, 'PiAnion.pkl'), 'rb') as file:
        PiAnion = pickle.load(file)
with open(Path.joinpath(data_dir, 'PiCation.pkl'), 'rb') as file:
        PiCation = pickle.load(file)

In [102]:
probar = [ pdb_idcode for pdb_idcode in pdb_list if len(PiCation[pdb_idcode]) > 3 ]

In [103]:
pdb_idcode = probar[0]
PiCation[pdb_idcode]

(PionPair(ring=Ring(indices=(708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721), serials=(709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722), resSeq=94, resSeq_str='', resname='TRP', chain_ID='A', chain_type='L', CDR=3), ion=Atom(index=3643, serial=6938, element='N', is_sidechain=True, resSeq=45, resSeq_str='45', resname='ARG', chain_ID='E', chain_type='.', CDR=-1)),
 PionPair(ring=Ring(indices=(1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908), serials=(1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910), resSeq=33, resSeq_str='', resname='TRP', chain_ID='B', chain_type='H', CDR=0), ion=Atom(index=3832, serial=7127, element='N', is_sidechain=True, resSeq=68, resSeq_str='68', resname='ARG', chain_ID='E', chain_type='.', CDR=-1)),
 PionPair(ring=Ring(indices=(1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908), serials=(1897, 1898, 1899, 1900, 1901, 190

In [147]:
def ions(atoms_iterable, resnames, element):
        control_set = set()
        for atom in atoms_iterable:
            if (atom.element == element) and (atom.resname in resnames) and atom.is_sidechain:
                # Get only 1 ion per residue (ASP/GLU or LYS/ARG)
                unique_id = str(atom.resSeq) + atom.resname + atom.chain_ID
                if unique_id not in control_set:
                    control_set.add(unique_id)
                    yield atom

In [156]:
retro = [ pdb_idcode for pdb_idcode in pdb_list if len(PiCation[pdb_idcode]) > 2]
pdb_idcode = retro[0]

In [157]:
a, b, c, d = get_ions(interface_atoms[pdb_idcode])

In [172]:
PiPi['5t1d']

()

In [92]:
def ions(atoms_iterable, resnames, element):
    # Get only 1 ion per residue (ASP/GLU or LYS/ARG)
    control_set = set()
    for atom in atoms_iterable:
        if (atom.element == element) and (atom.resname in resnames) and atom.is_sidechain:
            unique_id = str(atom.resSeq) + atom.resname + atom.chain_ID
            if unique_id not in control_set:
                control_set.add(unique_id)
                yield atom

Atom(index=3830, serial=7125, element='N', is_sidechain=True, resSeq=68, resSeq_str='68', resname='ARG', chain_ID='E', chain_type='.', CDR=-1)
Atom(index=3643, serial=6938, element='N', is_sidechain=True, resSeq=45, resSeq_str='45', resname='ARG', chain_ID='E', chain_type='.', CDR=-1)


In [109]:
gg = ions(interface_atoms[pdb_idcode].antigen.values(), {'LYS', 'ARG'}, 'N')


True

In [90]:
interface_atoms[pdb_idcode].antigen.values()

dict_values([Atom(index=573, serial=574, element='N', is_sidechain=False, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=574, serial=575, element='C', is_sidechain=False, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=575, serial=576, element='C', is_sidechain=False, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=576, serial=577, element='O', is_sidechain=False, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=577, serial=578, element='C', is_sidechain=True, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=578, serial=579, element='C', is_sidechain=True, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.', CDR=-1), Atom(index=579, serial=580, element='C', is_sidechain=True, resSeq=76, resSeq_str='76', resname='GLU', chain_ID='A', chain_type='.'

------

### hbond

In [4]:
import interactions_polar
importlib.reload(interactions_polar)
from interactions_polar import *

with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)

In [6]:
hbonds_dict = {}

check_pdb = '6edu'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
    print(f"{pdb_idcode}", flush=True)
    pdb_filename = Path.joinpath(str_dir, pdb_idcode + '.pdb')
    try:
        trj_in = md.load(pdb_filename)
    except Exception as e:
        logging.error(f" Couldn't read {pdb_idcode}. Skipping.")
        continue
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    hbond_dir = Path.joinpath(data_dir, "hbonds")
    hbplus = Path.joinpath(source_location, 'hbplus')

    process = subprocess.run([hbplus, pdb_filename, "-A", "0", "0", "0", "-d", "3.9"],
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE, cwd=hbond_dir)
    hb2_file = Path.joinpath(hbond_dir, pdb_filename.name[0:-3] + "hb2")

    hbonds_dict[pdb_idcode] = parse_hb2(
            pdb_idcode, hb2_file, epitope_buried_cleaned, trj_in.topology, ab_chains, ag_chains)

6edu


ERROR:root: Couldn't read 6edu. Skipping.


---

### shielding

In [7]:
import shielding
importlib.reload(shielding)
from shielding import *

with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)



In [ ]:
check_pdb = '5f96'
idx = pdb_list.index(check_pdb)
for pdb_idcode, pdb_file, is_cpx in zip([pdb_list[idx]], [file_pdb_list[idx]], [is_cpx_pdb[idx]]):
    print(f"{pdb_idcode}", flush=True)
    
    pdb_filename = Path.joinpath(casa_dir, pdb_file)
    trj_in = md.load(pdb_filename)

    all_ab_chains = [
        fila.chainID for i,
        fila in df_dataset[df_dataset.idcode == pdb_idcode].iterrows()]
    ag_chains = [
        chain.chain_id for chain in trj_in.topology.chains
        if chain.chain_id not in all_ab_chains]
    ab_chains = [
        chain.chain_id for chain in trj_in.topology.chains
        if chain.chain_id not in ag_chains]
        
    with open(Path.joinpath(casa_dir, 'data', 'hbonds_1_32.pkl'), 'rb') as file:
        hbonds_dict = pickle.load(file)

In [33]:
shielding_dict = {}
check_pdb = '1adq'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
    # for pdb_idcode in pdb_list:
    logging.info(pdb_idcode)

    pdb_filename = Path.joinpath(str_dir, pdb_idcode + '.pdb')
    try:
        trj_in = md.load(pdb_filename)
    except Exception as e:
        logging.error(f" Couldn't read {pdb_idcode}. Skipping.")
        continue
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    ab_carbons = [atom.index for atom in interface_atoms[pdb_idcode].antibody.values()
                if atom.element == 'C']

    ag_carbons = [atom.index for atom in interface_atoms[pdb_idcode].antigen.values()
                if atom.element == 'C']

    ab_polars = [atom.index for atom in interface_atoms[pdb_idcode].antibody.values()
                if atom.element in ('N', 'O')]
    ag_polars = [atom.index for atom in interface_atoms[pdb_idcode].antigen.values()
                if atom.element in ('N', 'O')]
    polars = ab_polars + ag_polars

    C_ON_pairs = np.array(list(itertools.product(ab_carbons, polars)))
    C_C_pairs = np.array(list(itertools.product(ab_carbons, ag_carbons)))

    C_ON_distancias = md.compute_distances(
        trj_in, C_ON_pairs).reshape((len(ab_carbons), len(polars)))

    C_C_distancias = md.compute_distances(
        trj_in, C_C_pairs).reshape((len(ab_carbons), len(ag_carbons)))

    G = nx.Graph()
    indices_close_C_C_distancias = np.where(C_C_distancias < cutoff)
    mask_close_C_ON_distancias = C_ON_distancias < cutoff
    shielding_pdb = {}
    for i, j in zip(*indices_close_C_C_distancias):
        C_cdr_id = ab_carbons[i]
        C_epi_id = ag_carbons[j]
        surrounding_ON_ids = [
            polars[i]
            for i in np.where(mask_close_C_ON_distancias[i, :])[0]]

        shielded, ON_id = is_shielded(
            trj_in.xyz[0], C_cdr_id, C_epi_id, surrounding_ON_ids)

        if shielded:
            # I could rewrite this to get the data from `interface_atoms[pdb_idcode]`
            # but I'm in a bit of a hurry.
            chainID = trj_in.topology.atom(ON_id).residue.chain.chain_id
            resSeq = trj_in.topology.atom(ON_id).residue.resSeq
            resname = trj_in.topology.atom(ON_id).residue.name
            chain_type, cdr = get_chain_info(
                epitope_buried_cleaned, pdb_idcode, ab_chains, chainID, resSeq)
            serial = trj_in.topology.atom(ON_id).serial
            element = trj_in.topology.atom(ON_id).element.symbol
            is_sidechain = trj_in.topology.atom(ON_id).is_sidechain

            # Compile all the info on this shielding polar atom.
            shielding_atom = ShieldingAtom(
                chainID=chainID, chain_type=chain_type, CDR=cdr,
                resSeq=resSeq, resname=resname, index=ON_id,
                serial=serial, element=element, is_sidechain=is_sidechain)

            shielding_pdb[ON_id] = shielding_atom

    shielding_dict[pdb_idcode] = shielding_pdb

In [22]:
with open(Path.joinpath(casa_dir, 'data', 'shielding.pkl'), 'rb') as file:
        shielding = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'hydrophobic.pkl'), 'rb') as file:
        hydrophobic = pickle.load(file)

In [11]:
pdb_idcode = '3ze1'
filenames[pdb_idcode]

'3ze1_complex_EF_C.pdb'

In [ ]:
hydro

-----

### Count_ONs

In [87]:
with open(Path.joinpath(casa_dir, 'data', 'hbonds_0_39.pkl'), 'rb') as file:
        hbonds = pickle.load(file)
interacting_ab_chains = {}

In [96]:
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
        filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'hbonds_0_39.pkl'), 'rb') as file:
    hbonds = pickle.load(file)

pdb_list = list(filenames.keys())
df_dataset = get_df_dataset(casa_dir)

interacting_ab_chains = {}
for pdb_idcode in pdb_list:
    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    serial_to_id = {}
    for atomo in trj_in.topology.atoms:
        serial_to_id[atomo.serial] = atomo.index

    ids_oxy_nitro = []
    for lista_hbond in hbonds[pdb_idcode].values():
        for hb in lista_hbond:
            ids_oxy_nitro.append(hb.donor.chainID)
            ids_oxy_nitro.append(hb.acceptor.chainID)

    interacting_ab_chains_pdb = set(Counter(ids_oxy_nitro).keys()) & set(ab_chains)

    interacting_ab_chains[pdb_idcode] = interacting_ab_chains_pdb

SabDab protein antigen:
1154 proteins out of 2017, 57.2%
All: 1154
No Hchain: 0
No Lchain: 0
Both chains: 1154
Buried surfaces of 2492 proteins
with both chains: 867


### DSSP completo

In [67]:
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
    filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'interacting_chains.pkl'), 'rb') as file:
    interacting_chains = pickle.load(file)
# with open(Path.joinpath(casa_dir, 'data', 'hbonds_0_39.pkl'), 'rb') as file:
#     hbonds = pickle.load(file)

pdb_list = list(filenames.keys())
df_dataset = get_df_dataset(casa_dir)

SabDab protein antigen:
1154 proteins out of 2017, 57.2%
All: 1154
No Hchain: 0
No Lchain: 0
Both chains: 1154
Buried surfaces of 2492 proteins
with both chains: 867


In [120]:
simplify_sse = {'H': 'H', 'B': 'E', 'E': 'E', 'G': 'H', 'I': 'H',
    'T': 'H', 'S': 'C', ' ': 'C', 'NA': 'NA'}
SSE = {}
SSE_cnt = {}
check_pdb = '1ncc'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
# for pdb_idcode in pdb_list:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ag_chains = chains[pdb_idcode].antigen
    # Get SSE of each residue
    dssp = md.compute_dssp(trj_in, simplified=False)[0]
    SSE_pdb = {}
    SSE_cnt_pdb = {'H': 0, 'E': 0, 'C': 0, 'NA': 0}
    for i, res in enumerate(trj_in.topology.residues):
        if res.chain.chain_id in ag_chains:
            sse = simplify_sse[dssp[i]]
            SSE_pdb[res.resSeq] = sse
            SSE_cnt_pdb[sse] += 1
    SSE[pdb_idcode] = SSE_pdb
    # SSE_cnt[pdb_idcode] = SSE_cnt_pdb

1ncc


In [121]:
Counter(SSE[pdb_idcode].values())

Counter({' ': 103, 'S': 62, 'B': 6, 'E': 162, 'H': 4, 'T': 42, 'G': 7})

----

### get_interface_atoms

In [72]:
import get_interface_atoms
imp.reload(get_interface_atoms)
from get_interface_atoms import *

In [50]:
with open(Path.joinpath(casa_dir, 'data', 'filenames.pkl'), 'rb') as file:
        filenames = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)
with(open(Path.joinpath(data_dir, 'pdb.list'), 'r')) as file:
    pdb_list = [linea.strip() for linea in file]

In [91]:
interface_atoms = {}
check_pdb = '1egj'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
# for pdb_idcode in pdb_list:
    

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    cadenas = get_chains(trj_in.topology, ab_chains, ag_chains)
    chain_types = get_chain_types(epitope_buried_cleaned, pdb_idcode, ab_chains, ag_chains)

    row_pdb = buried_interface_res.query(f"idcode == '{pdb_idcode}'\
        and chainID == '{ab_chains[0]}'")
    ab_atoms = []
    try:
        for row in row_pdb.ab_ag_interface_res.values[0]:
            chainID = row[0]
            resSeq_str = row[1].strip()
            resname = row[2]
            interface_residues = get_residue(cadenas[chainID], resSeq_str)
            for interface_resi in interface_residues:
                cdr = get_cdr_from_residue(
                    epitope_buried_cleaned, pdb_idcode, chainID, chain_types[chainID],
                    interface_resi)
                for atom in interface_resi.atoms:
                    atm = Atom(index=atom.index, serial=atom.serial,
                                resSeq=interface_resi.resSeq, resSeq_str=resSeq_str,
                                resname=interface_resi.name,
                                chain_ID=chainID, chain_type=chain_types[chainID], CDR=cdr)
                    ab_atoms.append(atm)

    except Exception as e:
        logging.exception(e)
        logging.error(f" {pdb_idcode} interface atom look-up failed.")
        continue

    ag_atoms = []
    try:
        for row in row_pdb.ag_ab_interface_res.values[0]:
            chainID = row[0]
            resSeq_str = row[1].strip()
            resname = row[2]
            interface_residues = get_residue(cadenas[chainID], resSeq_str)
            for interface_resi in interface_residues:
                cdr = get_cdr_from_residue(
                    epitope_buried_cleaned, pdb_idcode, chainID, chain_types[chainID],
                    interface_resi)
                for atom in interface_resi.atoms:
                    atm = Atom(index=atom.index, serial=atom.serial,
                                resSeq=interface_resi.resSeq, resSeq_str=resSeq_str,
                                resname=interface_resi.name,
                                chain_ID=chainID, chain_type=chain_types[chainID], CDR=cdr)
                    ag_atoms.append(atm)

    except Exception as e:
        logging.exception(e)
        logging.error(f" {pdb_idcode} interface atom look-up failed.")
        continue


---

### get_cdrs

In [8]:
cdr_residues = {}
KtoL = {'H': 'H', 'L': 'L', 'K': 'L'}
letras = dict(zip(range(0, len(string.ascii_uppercase)),
                    tuple(string.ascii_uppercase)))
check_pdb = '7k8u'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
    # for pdb_idcode in pdb_list:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    try:
        cdr_residues_pdb = {}

        for row in epitope_buried_cleaned.query(
                f"idcode == '{pdb_idcode}'").iterrows():

            chainID = row[1].chainID
            if chainID not in ab_chains:
                continue
            chain_type = KtoL[row[1].chain_type]
            cdr = int(row[1].cdr)
            cdr_id = chain_type + str(cdr)
            # Deal with characters in `cdr_begin` and `cdr_begin`:
            try:
                beg = int(row[1].cdr_begin)
            except ValueError:
                beg = int(row[1].cdr_begin[:-1])
            try:
                end = int(row[1].cdr_end)
            except ValueError:
                end = int(row[1].cdr_end[:-1])
            cdr_range = range(beg, end)

            

            # Get the list of CDR residues as MDtraj residues
            cderes = tuple([residue for residue in trj_in.topology.residues if (
                beg <= residue.resSeq <= end) and (residue.chain.chain_id == chainID)])

            # Now convert the CDR MDtraj residues to my residue format
            list_resis = []
            prev_residue = cderes[0]
            list_resis.append(
                Residue(
                    index=prev_residue.index, resSeq=prev_residue.resSeq,
                    resSeq_str=str(prev_residue.resSeq),
                    resname=prev_residue.name, chain_ID=chainID,
                    chain_type=chain_type, CDR=cdr, SSE=''))
            letter_counter = 0
            for residue in cderes[1:]:
                resSeq = residue.resSeq
                prev_resSeq = prev_residue.resSeq
                if resSeq == prev_resSeq:
                    resSeq_str = str(residue.resSeq) + letras[letter_counter]
                    letter_counter += 1
                else:
                    resSeq_str = str(residue.resSeq)
                    prev_residue = residue

                list_resis.append(
                    Residue(
                        index=residue.index, resSeq=resSeq, resSeq_str=resSeq_str,
                        resname=residue.name, chain_ID=chainID,
                        chain_type=chain_type, CDR=cdr, SSE=''))
            cdr_residues_pdb[cdr_id] = tuple(list_resis)
    except Exception as e:
        logging.exception(e)
        logging.error(f" {pdb_idcode} CDRs residues look-up failed.")
        continue

    cdr_residues[pdb_idcode] = cdr_residues_pdb

NameError: name 'string' is not defined

In [4]:
with open(Path.joinpath(casa_dir, 'data', 'cdr_residues.pkl'), 'rb') as file:
    cdr_residues = pickle.load(file)

In [6]:
with open(Path.joinpath(casa_dir, 'data', 'surface_residues.pkl'), 'rb') as file:
        surface_residues = pickle.load(file)

----

### freesasa

`structures/exposed/1egj/1egj_complex_HL_A.pdb` y otros tienen missings en el CDR H3

In [4]:
with open(Path.joinpath(casa_dir, 'data', 'shielding.pkl'), 'rb') as file:
    shielding_dict = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_polar.pkl'), 'rb') as file:
        count_polar = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_hbonds.pkl'), 'rb') as file:
        polar_hbonds = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_saltBridge.pkl'), 'rb') as file:
        saltBridge = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_wat.pkl'), 'rb') as file:
        polar_wat = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'cdr_residues.pkl'), 'rb') as file:
        cdr_residues = pickle.load(file)

In [22]:
surface_cdrh3 = {}
check_pdb = '1egj'
idx = pdb_list.index(check_pdb)
for pdb_idcode in [pdb_list[idx]]:
# for pdb_idcode in pdb_list:
    logging.info(pdb_idcode)

    pdb_filename = Path(filenames[pdb_idcode])
    pdb_file = Path.joinpath(exposed_dir, pdb_idcode, pdb_filename)
    # trj_in = md.load(Path.joinpath(exposed_dir, pdb_idcode, pdb_filename))
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen
    seleccion = "h3, resi " + '+'.join(
        [str(resi.resSeq) for resi in cdr_residues[pdb_idcode]['H3']])

    try:
        # structura = freesasa.Structure(str(pdb_file))
        # Separate chains to get the SASA of the free CDR H3
        structuras = freesasa.structureArray(
            str(pdb_file), {'separate-chains': True})

        for struct in structuras:
            
            if struct.chainLabel(0) == ab_chains[0]:
                resultado = freesasa.calc(struct)
                h3 = freesasa.selectArea([seleccion], struct, resultado)['h3']
                break
        else:
            raise Exception("Couldn't match heavy-chain.")
        surface_cdrh3[pdb_idcode] = h3

    except Exception as e:
        logging.exception(e)
        logging.error(f" {pdb_idcode} CDR H3 surface calculation failed.")
        continue

FreeSASA: warning: Found no matches to resi '100', typo?
FreeSASA: warning: Found no matches to resi '100', typo?
FreeSASA: warning: Found no matches to resi '100', typo?


In [25]:
with open(Path.joinpath(casa_dir, 'data', 'surface_cdrh3.pkl'), 'rb') as file:
        surface_cdrh3 = pickle.load(file)

------------

#### busco ejemplo de SER bordeando cluster

In [5]:
with open(Path.joinpath(casa_dir, 'data', 'shielding.pkl'), 'rb') as file:
    shielding_dict = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'count_polar.pkl'), 'rb') as file:
        count_polar = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_hbonds.pkl'), 'rb') as file:
        polar_hbonds = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_saltBridge.pkl'), 'rb') as file:
        saltBridge = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'polar_wat.pkl'), 'rb') as file:
        polar_wat = pickle.load(file)
with open(Path.joinpath(data_dir, "wat_pdb.list"), 'r') as file:
    wat_pdb = file.read().splitlines()

In [6]:
carucha = []
for pdb_idcode in pdb_list:
    if pdb_idcode in ('6edu', '6ss6', '7q9k'):
            continue
    
    hbond_list_resSeq_chainID_ag = []
    hbond_list_resSeq_chainID_ab = []
    for bond_list in polar_hbonds[pdb_idcode].values():
            for Htuple in bond_list:
                for h_atom in Htuple:
                    if h_atom.resname != 'SER':
                        continue
                    if h_atom.CDR == -1:
                        hbond_list_resSeq_chainID_ag.append(
                            h_atom.resname + h_atom.chainID + str(h_atom.resSeq))
                    else:
                        hbond_list_resSeq_chainID_ab.append(
                            h_atom.resname + h_atom.chainID + str(h_atom.resSeq))
    set_hbond_residues_ab = set(hbond_list_resSeq_chainID_ab)
    carucha.append((len(set_hbond_residues_ab), pdb_idcode))

In [7]:
lal = sorted(carucha, key = lambda x: x[0], reverse = True)
lal

[(7, '3so3'),
 (7, '7lcv'),
 (6, '5vk2'),
 (6, '6p95'),
 (6, '7lab'),
 (6, '7lbg'),
 (5, '3v6z'),
 (5, '5vlp'),
 (5, '7ket'),
 (5, '7or9'),
 (5, '7sem'),
 (4, '1i9r'),
 (4, '1ynt'),
 (4, '2nz9'),
 (4, '2r56'),
 (4, '2xqb'),
 (4, '3bn9'),
 (4, '3s37'),
 (4, '4k8r'),
 (4, '4kvn'),
 (4, '4o9h'),
 (4, '4yqx'),
 (4, '4yzf'),
 (4, '5b3j'),
 (4, '5f3b'),
 (4, '5vqm'),
 (4, '5w1k'),
 (4, '6apb'),
 (4, '6apd'),
 (4, '6hga'),
 (4, '6jjp'),
 (4, '6jod'),
 (4, '6q0i'),
 (4, '6wh9'),
 (4, '7czq'),
 (4, '7e8c'),
 (4, '7eya'),
 (4, '7m30'),
 (4, '7mjl'),
 (4, '7mxl'),
 (4, '7mzn'),
 (4, '7nd5'),
 (4, '7pi2'),
 (4, '7pi3'),
 (4, '7ps1'),
 (4, '7q0i'),
 (4, '7r8l'),
 (4, '7s5p'),
 (3, '1qgc'),
 (3, '1ztx'),
 (3, '2eiz'),
 (3, '2xwt'),
 (3, '3g6j'),
 (3, '3hi6'),
 (3, '3kr3'),
 (3, '3p0y'),
 (3, '3vg9'),
 (3, '3vrl'),
 (3, '4cad'),
 (3, '4dtg'),
 (3, '4hf5'),
 (3, '4lvo'),
 (3, '4m7l'),
 (3, '4n90'),
 (3, '4od2'),
 (3, '4qti'),
 (3, '4wfe'),
 (3, '4xzu'),
 (3, '4yfl'),
 (3, '4zxb'),
 (3, '5bvp'),
 (3, '

In [8]:
pdb_idcode = '5vk2'
filenames[pdb_idcode]

'5vk2_complex_DE_cC.pdb'

----

### tamaño de epitope & paratope

In [4]:
import get_interface_atoms
imp.reload(get_interface_atoms)
from get_interface_atoms import *

In [3]:
print("Reading buried_interface_res.pickle")
with open(Path.joinpath(
            casa_dir, "data", 'buried_interface_res.pickle'), 'rb') as file:
        buried_interface_res = pickle.load(file)
print("Reading epitope_buried_cleaned.pickle")
with open(Path.joinpath(
        casa_dir, "data", 'epitope_buried_cleaned.pickle'), 'rb') as file:
    epitope_buried_cleaned = pickle.load(file)
with open(Path.joinpath(casa_dir, 'data', 'chains.pkl'), 'rb') as file:
    chains = pickle.load(file)

Reading buried_interface_res.pickle
Reading epitope_buried_cleaned.pickle


In [13]:
# pdb_idcode = random.choice(pdb_list)
paratope_size = []
epitope_size = []
paratope_size_avg = []
epitope_size_avg = []
for pdb_idcode in pdb_list:
    
    ab_chains = chains[pdb_idcode].antibody
    ag_chains = chains[pdb_idcode].antigen

    df_interface_atoms_avg = buried_interface_res.query(f"idcode == '{pdb_idcode}'")
    interfaz = next(df_interface_atoms_avg.iterrows())
        # atms = next(iter(df_interface_atoms.ab_ag_interface_res))
    unique_res_ab = { atom[0:3] for atom in interfaz[1].ab_ag_interface_res }
    paratope_size_avg.append(len(unique_res_ab))

    # atms = next(iter(df_interface_atoms.ag_ab_interface_res))
    unique_res_ag = { atom[0:3] for atom in interfaz[1].ag_ab_interface_res }
    epitope_size_avg.append(len(unique_res_ag))

    # `buried_interface_res` has 1 row per interface. In case there's more than one,
    # I use the heavy chain (ab_chains[0]) to identify the one I'm care about.
    df_interface_atoms = buried_interface_res.query(f"idcode == '{pdb_idcode}'\
            and chainID == '{ab_chains[0]}'")

    atms = next(iter(df_interface_atoms.ab_ag_interface_res))
    unique_res_ab = { atom[0:3] for atom in atms }
    paratope_size.append(len(unique_res_ab))

    atms = next(iter(df_interface_atoms.ag_ab_interface_res))
    unique_res_ag = { atom[0:3] for atom in atms }
    epitope_size.append(len(unique_res_ag))

np.mean(paratope_size), np.std(paratope_size),\
    np.mean(epitope_size), np.std(epitope_size)

(18.68968253968254, 5.146864191945428, 17.49920634920635, 5.489889639966118)

In [86]:
paratope_size_avg = []
epitope_size_avg = []
for row in buried_interface_res.iterrows():
      
    unique_res_ab = { atom[0:3] for atom in row[1].ab_ag_interface_res }
    paratope_size_avg.append(len(unique_res_ab))
    
    unique_res_ag = { atom[0:3] for atom in row[1].ag_ab_interface_res }
    epitope_size_avg.append(len(unique_res_ag))

print(f"Paratope size: {np.round(np.mean(paratope_size_avg), 2)}")
print(f"Epitope size: {np.round(np.mean(epitope_size_avg), 2)}")

Paratope size: 18.32
Epitope size: 17.07


In [81]:
# np.round(np.std(paratope_size_avg), 2),
# np.round(np.std(epitope_size_avg), 2)

{('O', '3033 ', 'PRO'),
 ('O', '3050 ', 'LEU'),
 ('O', '3052 ', 'TYR'),
 ('O', '3055 ', 'THR'),
 ('O', '3057 ', 'GLU'),
 ('O', '3102 ', 'ILE'),
 ('P', '3531 ', 'SER'),
 ('P', '3532 ', 'SER'),
 ('P', '3534 ', 'CYS'),
 ('P', '3593 ', 'TRP'),
 ('P', '3595 ', 'ASN'),
 ('P', '3596 ', 'ASN'),
 ('P', '3598 ', 'TRP')}

In [14]:
np.mean(paratope_size_avg), np.std(paratope_size_avg),\
    np.mean(epitope_size_avg), np.std(epitope_size_avg)

(18.35952380952381, 5.185000934800065, 17.1015873015873, 5.458137715534806)